## Calculate Mesozooplankton trophic level 

In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import os
from glob import glob
from collections import OrderedDict
import xarray as xr
import numpy as np
import ncar_jobqueue
from dask.distributed import Client

import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
from scipy import stats
import pop_tools 

import cftime
import pandas as pd
import utils

import cmocean

In [2]:
case = 'g.e21.G1850ECOIAF.t62_g17.marbl0_33.GNG595'
path = f'/glade/campaign/cesm/development/bgcwg/projects/marbl-spectra/{case}/ocn/hist'

In [3]:
## Campaign 2nd Cycle 
campaign_20yr = '/glade/campaign/cesm/development/bgcwg/projects/marbl-spectra/GNG595_2ndcycle_1990-2009_clim/GNG595_clim_1990-2009'
campaign_full = '/glade/campaign/cesm/development/bgcwg/projects/marbl-spectra/g.e21.G1850ECOIAF.t62_g17.marbl0_33.GNG595/ocn/hist/'
newNCDFfiles1 = sorted(glob(f'{campaign_20yr}_*.nc', recursive=True),
                      key=utils.numericalSort)

ds = xr.open_mfdataset(newNCDFfiles1,concat_dim='time', combine='nested')

In [4]:
mesozoo_names = { 'mesozoo_names': ['zoo3','zoo4','zoo5','zoo6']}
zn = pd.DataFrame(data=mesozoo_names)

phyto_names = { 'phyto_names': ['pp','mp1','diaz','mp2','diat1','diat2','mp3','diat3','mp4']}
pn = pd.DataFrame(data=phyto_names)

## Get plankton size information from file.

In [5]:
## Gaby's Path 
gabyn_path = '/glade/work/gabyn/case_notes/spectral_cases/g.e21.G1850ECOIAF.t62_g17.marbl0_33.'
nl_config = os.path.join(''+gabyn_path+'GNG595/data/')
sizes = pd.read_csv(nl_config+'plankton_sizes.csv')
sizes = sizes.sort_values('mmolC')

sizes

,type,sname,mass_ugC,vol_um3,ESD_mm,Qp_fixed,mmolC
1,phyto,pp,9.266989e-08,3.706795e-01,0.000891,0.004651,7.715418e-12
2,phyto,mp1,8.241979e-06,1.805089e+01,0.003255,0.006824,6.862025e-10
0,phyto,diaz,5.499929e-05,1.227865e+02,0.006167,0.003333,4.579077e-09
3,phyto,mp2,8.888712e-05,8.790197e+02,0.011885,0.008656,7.400476e-09
6,phyto,diat1,5.004513e-04,1.327390e+04,0.029376,0.010289,4.166608e-08
9,zoo,zoo1,2.803686e-03,2.355042e+04,0.035563,0.008547,2.334265e-07
7,phyto,diat2,3.803133e-03,1.326626e+05,0.063278,0.012602,3.166375e-07
4,phyto,mp3,4.830334e-03,4.280541e+04,0.043401,0.012907,4.021592e-07
8,phyto,diat3,2.890156e-02,1.325863e+06,0.136301,0.015435,2.406258e-06
10,zoo,zoo2,8.933984e-02,7.447577e+05,0.112462,0.008547,7.438168e-06


In [6]:
prey_varnames = {}
for i,v in enumerate(sizes.sname):
    prey_varnames[v]=i

In [7]:
pred_varnames = {'zoo3':2,'zoo4':3,'zoo5':4,'zoo6':5}
grazing_vars = [v for v in ds if 'graze' in v and 'zint' not in v]

In [8]:
graze_split = [v.split('_',3) for v in grazing_vars]
graze_split = pd.DataFrame(graze_split)
graze_split = graze_split.rename({0:'var',1:'prey',2:'pred'}, axis=1)
graze_split['var'] = grazing_vars

In [9]:
variables = [v for v in ds if 'graze' in v and 'zint' not in v]
coords = {'x':'TLONG','y':'TLAT'}
keep_vars = variables + list(coords.values())+['dz','KMT','time']

In [11]:
%%time

ds_avg = xr.Dataset()

for year in np.arange(63,125,1):
    yr4="0{:02d}".format(year).zfill(4)
    print(year)
    
    ds_annual = xr.Dataset()

    file = sorted(glob(f'{path}/{case}.pop.h.{yr4}-*.nc'))
    
    dsv_annual=xr.open_mfdataset(file, decode_times=True,drop_variables=["transport_components", "transport_regions"], 
                            parallel=True, compat="override", combine='nested', concat_dim="time",data_vars="minimal",coords='minimal' )
    
    for vv in variables: 
        ds_annual = xr.merge((ds_annual, dsv_annual[vv]))
        

    ds_annual = ds_annual.drop([v for v in ds_annual.variables if v not in keep_vars]).squeeze()
    ds_annual = ds_annual.mean(dim='time')
    ds_avg = xr.concat([ds_avg, ds_annual],dim='year')

63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
CPU times: user 9min 43s, sys: 5min 9s, total: 14min 52s
Wall time: 15min 47s


In [12]:
ds_avg

,Array,Chunk
Bytes,435.94 MiB,7.03 MiB
Shape,"(62, 15, 384, 320)","(1, 15, 384, 320)"
Count,1859 Graph Layers,62 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,435.94 MiB,7.03 MiB
Shape,"(62, 15, 384, 320)","(1, 15, 384, 320)"
Count,1859 Graph Layers,62 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


## Get a cluster so it can continue running without interruptions

In [13]:
USER = os.environ['USER']
def get_ClusterClient():
    import dask
    from dask_jobqueue import PBSCluster
    from dask.distributed import Client
    cluster = PBSCluster(
        cores=2,
        memory='25 GB',
        processes=1,
        queue='casper',
        resource_spec='select=1:ncpus=1:mem=25GB',
        project='P93300070',
        walltime='12:00:00',
        interface='ib0',)

    dask.config.set({
        'distributed.dashboard.link':
        'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'
    })
    client = Client(cluster)
    return cluster, client

In [14]:
cluster, client = get_ClusterClient()
cluster.scale(10) 
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/gabyn/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/gabyn/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.46:39141,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/gabyn/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [15]:
ds_avg.graze_diaz_zoo3.shape

(62, 15, 384, 320)

In [16]:
# set trophic level for phyto = 1
tmpOnes = np.ones(shape=ds_avg.graze_diaz_zoo3.shape)
tlvals = {}
for v in pn.phyto_names:
    tlvals[v]=tmpOnes

# set trophic level for zoo = NaN initially
tmp = tmpOnes.copy()
tmp[:] = np.nan
for v in zn.mesozoo_names:
    tlvals[v]=tmp
    
zoo = ['zoo3','zoo4','zoo5','zoo6']

In [17]:
import re
numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

In [18]:
prey_varnames = {}
for i,v in enumerate(sizes.sname):
    prey_varnames[v]=i

In [19]:
pred_varnames = {'zoo3':2,'zoo4':3,'zoo5':4,'zoo6':5}
grazing_vars = [v for v in ds if 'graze' in v and 'zint' not in v]

In [20]:
grazing_vars

['graze_diaz_zoo1',
 'graze_diaz_zoo2',
 'graze_diaz_zoo3',
 'graze_pp_zoo1',
 'graze_mp1_zoo1',
 'graze_mp1_zoo2',
 'graze_mp2_zoo2',
 'graze_mp2_zoo3',
 'graze_mp2_zoo4',
 'graze_mp3_zoo3',
 'graze_mp3_zoo4',
 'graze_mp3_zoo5',
 'graze_mp4_zoo4',
 'graze_mp4_zoo5',
 'graze_mp4_zoo6',
 'graze_diat1_zoo1',
 'graze_diat1_zoo2',
 'graze_diat1_zoo3',
 'graze_diat1_zoo4',
 'graze_diat2_zoo2',
 'graze_diat2_zoo3',
 'graze_diat2_zoo4',
 'graze_diat2_zoo5',
 'graze_diat3_zoo3',
 'graze_diat3_zoo4',
 'graze_diat3_zoo5',
 'graze_diat3_zoo6',
 'graze_zoo1_zoo2',
 'graze_zoo1_zoo3',
 'graze_zoo2_zoo3',
 'graze_zoo2_zoo4',
 'graze_zoo3_zoo4',
 'graze_zoo3_zoo5',
 'graze_zoo4_zoo5',
 'graze_zoo4_zoo6',
 'graze_zoo5_zoo6']

In [21]:
%%time
for i,vs in enumerate(zn.mesozoo_names):
    
    # pull out the appropriate grazing variables in the history file
    graze_vars = [v for v in ds_avg if re.match('graze_(.*)_'+vs,v)]
    
    # holder for the grazing values
    shape = ds_avg[graze_vars[0]].shape + (len(graze_vars),)

    graze_vals = np.empty(shape=shape)
    graze_tl = np.ones(shape=shape)
    
    for gi,gv in enumerate(graze_vars):
        
        # pull out identity of prey
        g,prey,pred = gv.split('_',3)
        print(prey,pred)
        
        # if prey is zooplankton, then apply alternate trophic level
        if prey in zoo:
            graze_tl[...,gi]=tlvals[prey]
        
        graze_vals[...,gi] = ds_avg[gv].values # Ellipses select the last slice
    # graze_vals*grazetl/graze_vals    
    tlvals[vs] = 1 + (np.sum(graze_vals * graze_tl, axis=-1) / np.sum(graze_vals, axis=-1)) # axis=-1 means sum over the last axis

diaz zoo3
mp2 zoo3
mp3 zoo3
diat1 zoo3
diat2 zoo3
diat3 zoo3
zoo1 zoo3
zoo2 zoo3
mp2 zoo4
mp3 zoo4
mp4 zoo4
diat1 zoo4
diat2 zoo4
diat3 zoo4
zoo2 zoo4
zoo3 zoo4
mp3 zoo5
mp4 zoo5
diat2 zoo5
diat3 zoo5
zoo3 zoo5
zoo4 zoo5
mp4 zoo6
diat3 zoo6
zoo4 zoo6
zoo5 zoo6
CPU times: user 4min 35s, sys: 6min 35s, total: 11min 10s
Wall time: 45min 43s


In [22]:
%%time
np.save("/glade/scratch/gabyn/SPECTRA/tlvals_62yr.npy", tlvals)

CPU times: user 1.85 s, sys: 8.68 s, total: 10.5 s
Wall time: 36 s


In [23]:
%%time
zoods = xr.DataArray(tlvals['zoo3'], coords={'year':ds_avg.year, 'z_t_150m':ds_avg.z_t_150m, 'nlat':ds_avg.nlat, 'nlon':ds_avg.nlon},
                      dims=['year', 'z_t_150m', 'nlat', 'nlon'])
zoods = zoods.to_dataset(name='zoo3TL')

zoods['zoo4TL'] = xr.DataArray(tlvals['zoo4'], coords={'year':ds_avg.year, 'z_t_150m':ds_avg.z_t_150m, 'nlat':ds_avg.nlat, 'nlon':ds_avg.nlon},
                      dims=['year', 'z_t_150m', 'nlat', 'nlon'])
zoods['zoo5TL'] = xr.DataArray(tlvals['zoo5'], coords={'year':ds_avg.year, 'z_t_150m':ds_avg.z_t_150m, 'nlat':ds_avg.nlat, 'nlon':ds_avg.nlon},
                      dims=['year', 'z_t_150m', 'nlat', 'nlon'])
zoods['zoo6TL'] = xr.DataArray(tlvals['zoo6'], coords={'year':ds_avg.year, 'z_t_150m':ds_avg.z_t_150m, 'nlat':ds_avg.nlat, 'nlon':ds_avg.nlon},
                      dims=['year', 'z_t_150m', 'nlat', 'nlon'])

CPU times: user 5.03 ms, sys: 0 ns, total: 5.03 ms
Wall time: 5.83 ms


In [24]:
%%time
zoods['zooTL_g'] = xr.concat([zoods[v+'TL'] for v in ['zoo3','zoo4','zoo5','zoo6']],dim='var').mean(dim='var')

CPU times: user 12.2 s, sys: 24 s, total: 36.2 s
Wall time: 2min 4s


In [25]:
zoods

<xarray.Dataset>
Dimensions:   (year: 62, z_t_150m: 15, nlat: 384, nlon: 320)
Coordinates:
  * year      (year) int64 0 1 2 3 4 5 6 7 8 9 ... 52 53 54 55 56 57 58 59 60 61
  * z_t_150m  (z_t_150m) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14
  * nlat      (nlat) int64 0 1 2 3 4 5 6 7 8 ... 376 377 378 379 380 381 382 383
  * nlon      (nlon) int64 0 1 2 3 4 5 6 7 8 ... 312 313 314 315 316 317 318 319
Data variables:
    zoo3TL    (year, z_t_150m, nlat, nlon) float64 nan nan nan ... nan nan nan
    zoo4TL    (year, z_t_150m, nlat, nlon) float64 nan nan nan ... nan nan nan
    zoo5TL    (year, z_t_150m, nlat, nlon) float64 nan nan nan ... nan nan nan
    zoo6TL    (year, z_t_150m, nlat, nlon) float64 nan nan nan ... nan nan nan
    zooTL_g   (year, z_t_150m, nlat, nlon) float64 nan nan nan ... nan nan nan

In [28]:
np.save("/glade/scratch/gabyn/SPECTRA/GNG595_npy_files/mesozoo_TL_62yr.npy", zoods.zooTL_g)

In [30]:
%%time
np.save("/glade/scratch/gabyn/SPECTRA/GNG595_npy_files/zoo3TL_62yr.npy", zoods.zoo3TL)
np.save("/glade/scratch/gabyn/SPECTRA/GNG595_npy_files/zoo4TL_62yr.npy", zoods.zoo4TL)
np.save("/glade/scratch/gabyn/SPECTRA/GNG595_npy_files/zoo5TL_62yr.npy", zoods.zoo5TL)
np.save("/glade/scratch/gabyn/SPECTRA/GNG595_npy_files/zoo6TL_62yr.npy", zoods.zoo6TL)

CPU times: user 1.21 s, sys: 3.52 s, total: 4.73 s
Wall time: 24 s


## Once done, close the cluster

In [31]:
cluster.close()